In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.applications.efficientnet as efn

In [24]:
# train_datagen = ImageDataGenerator(
#     rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
#     rotation_range=20,  # Randomly rotate images by 20 degrees
#     width_shift_range=0.2,  # Randomly shift the width by 20%
#     height_shift_range=0.2,  # Randomly shift the height by 20%
#     horizontal_flip=True  # Randomly flip images horizontally
# )

In [25]:
# valid_datagen = ImageDataGenerator(
#     rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
#     rotation_range=20,  # Randomly rotate images by 20 degrees
#     width_shift_range=0.2,  # Randomly shift the width by 20%
#     height_shift_range=0.2,  # Randomly shift the height by 20%
#     horizontal_flip=True  # Randomly flip images horizontally
# )

In [26]:

# test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [27]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN'
valid_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID'
test_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST'

train_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224, 224), batch_size=32)


Found 17640 images belonging to 15 classes.


In [28]:


valid_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

Found 5040 images belonging to 15 classes.


In [29]:
test_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224, 224), batch_size=32,shuffle = False)

Found 2520 images belonging to 15 classes.


In [30]:
import tensorflow as tf
efficient = tf.keras.applications.EfficientNetB0(include_top = False,input_shape=(224,224,3))
efficient.summary()
     
     

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 224, 224, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_2[0][0]            
_____________________________________________________________________________________

In [31]:
for layer in efficient.layers[:-75]:
    layer.trainable = False
    print(layer.name)

input_3
rescaling_2
normalization_2
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block3a_expand_conv
block3a_expand_bn
block3a_expand_activation
block3a_dwconv_pad
block3a_dwconv
block3a_bn
block3a_activation
block3a_se_squeeze
block3a_se_reshape
block3a_se_reduce
block3a_se_expand
block3a_se_

In [32]:

from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

x = efficient.layers[-1].output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(1024,activation = 'relu')(x)
output = tf.keras.layers.Dense(15,activation = 'softmax')(x)

model = Model(inputs=efficient.input, outputs=output)

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 224, 224, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_2[0][0]            
______________________________________________________________________________________________

In [34]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=10,
            verbose=1
)

Epoch 1/10
552/552 [==============================] - 1511s 3s/step - loss: 1.5875 - accuracy: 0.4970 - val_loss: 1.0934 - val_accuracy: 0.6520
Epoch 2/10
552/552 [==============================] - 1534s 3s/step - loss: 0.9909 - accuracy: 0.6769 - val_loss: 0.9423 - val_accuracy: 0.6968
Epoch 3/10
552/552 [==============================] - 1479s 3s/step - loss: 0.7483 - accuracy: 0.7571 - val_loss: 0.9031 - val_accuracy: 0.7202
Epoch 4/10
552/552 [==============================] - 1479s 3s/step - loss: 0.5622 - accuracy: 0.8175 - val_loss: 0.9133 - val_accuracy: 0.7192
Epoch 5/10
552/552 [==============================] - 1470s 3s/step - loss: 0.4120 - accuracy: 0.8660 - val_loss: 0.9360 - val_accuracy: 0.7331
Epoch 6/10
552/552 [==============================] - 1600s 3s/step - loss: 0.3074 - accuracy: 0.9001 - val_loss: 0.9937 - val_accuracy: 0.7282
Epoch 7/10
552/552 [==============================] - 1521s 3s/step - loss: 0.2353 - accuracy: 0.9247 - val_loss: 1.0407 - val_accuracy:

In [40]:
for layer in model.layers:
    layer.trainable = True
    
optimizer = Adam(learning_rate = 0.00007)
model.compile(optimizer = optimizer,loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [41]:
model.fit(x = train_batches,
        steps_per_epoch = len(train_batches),
        validation_data = valid_batches,
        validation_steps = len(valid_batches),
        epochs = 12,
        verbose = 1
)

Epoch 1/12
552/552 [==============================] - 4613s 8s/step - loss: 0.5913 - accuracy: 0.8087 - val_loss: 0.9326 - val_accuracy: 0.7478
Epoch 2/12
552/552 [==============================] - 4182s 8s/step - loss: 0.2641 - accuracy: 0.9139 - val_loss: 0.9857 - val_accuracy: 0.7556
Epoch 3/12
552/552 [==============================] - 4487s 8s/step - loss: 0.1733 - accuracy: 0.9427 - val_loss: 1.0433 - val_accuracy: 0.7530
Epoch 4/12
 10/552 [..............................] - ETA: 1:07:44 - loss: 0.1074 - accuracy: 0.9594

KeyboardInterrupt: 

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [43]:
test_labels = test_batches.classes

In [44]:
predictions = model.predict(x=test_batches)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the testing accuracy
testing_accuracy = np.mean(predicted_labels == test_labels)

print("Testing Accuracy:", testing_accuracy)

Testing Accuracy: 0.7587301587301587


In [ ]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [ ]:

test_batches.class_indices

In [ ]:
cm_plot_labels = ['sitting','using_laptop','hugging','sleeping','drinking','clapping','dancing','cycling','calling','laughing','eating','fighting','listening_to_music','running','texting']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
model.save_weights('KerasEfficientNetB0_aug.h5')

In [ ]:

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
TF_LITE_MODEL_FILE_NAME = "KerasEfficientNetB0_aug.tflite"

In [ ]:

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
import os
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")
     